In [34]:
import os
import json 
from dotenv import load_dotenv

from typing import List

import openai
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

from langchain_core.tools import tool
from pydantic import BaseModel, Field

import httpx 
import smtplib
from email.mime.text import MIMEText
from email.message import EmailMessage

In [35]:
load_dotenv()
os.environ['groq_api_key'] = os.getenv('groq_api_key')
jina_api_key = os.getenv('jina_api_key')
password = os.getenv('app_password')
email = os.getenv('email')


# llm = ChatOpenAI(model='gpt-4o-mini')
# llm = ChatGroq(model='llama-3.3-70b-versatile')

In [36]:
class Email(BaseModel):
  subject: str = Field(description="Subject of the email")
  body: str = Field(description="Body of the email")


@tool
def scrape_website(url: str):
    """use to get the text from a website
    Args:
        url: url to scrape
    """
    headers = {'Authorization': f'Bearer {jina_api_key}'}
    response = httpx.get(f'https://r.jina.ai/{url}', headers=headers)
    return response.text


@tool
def write_newsletter(text: str):
    """used to write a newsletter from the provided text
    Args:
        text: text to write the newsletter from
    """
    completion = client.beta.chat.completions.parse(
        model='gpt-4o-mini',
        messages=[
            {'role': 'system', 'content': 'You"re an expert at writing email newsletters. Use the provided context to write a newsletter to a general audience. remove all links and images present in it'},
            {'role': 'user', 'content': f'context: {text}'},
        ],
        response_format=Email,
    )
    data = json.dumps(completion.choices[0].message.parsed.dict())
    data = json.loads(data)

    new_dict = {}
    new_dict['Subject'] = data['subject']
    new_dict['Body'] = data['body']
    return new_dict


@tool
def send_mail(subject, body):
    """sends email to user(s).
    Args:
        subject: subject of email
        body: body of the email
        recipients: list of email address
    """
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(emails)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
        smtp_server.login(sender, password)
        smtp_server.sendmail(sender, emails, msg.as_string())
    return "Message sent!"

In [37]:
client = openai.OpenAI()
sender = email
password = password
emails = ["adejumobidaniel563@gmail.com", "sunnyadejumobi@gmail.com"]
url = 'https://paulgraham.com/startupideas.html'


system_prompt = """
You are an expert at writing newsletters, and your task is to use the given context 
to write email newsletters and send it to appropriate recipients

These tools are available to you:

- scrape_website: Use to get the text from a website
- write_newsletter: Use to write a newsletter from the provided text
- send_mail: Sends email to multiple users
"""

user_prompt = f"""
system prompt: 
{system_prompt}

User prompt: 
write a newsletter using text from {url} and send it to the following emails {emails}
"""

print(user_prompt)


system prompt: 

You are an expert at writing newsletters, and your task is to use the given context 
to write email newsletters and send it to appropriate recipients

These tools are available to you:

- scrape_website: Use to get the text from a website
- write_newsletter: Use to write a newsletter from the provided text
- send_mail: Sends email to multiple users


User prompt: 
write a newsletter using text from https://paulgraham.com/startupideas.html and send it to the following emails ['adejumobidaniel563@gmail.com', 'sunnyadejumobi@gmail.com']



In [38]:
tools = [scrape_website, write_newsletter, send_mail]
llms_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}

def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls

chain = llms_with_tools | call_tools

In [39]:
response = chain.invoke(user_prompt)

C:\Users\HomePC\AppData\Local\Temp\ipykernel_14548\1977018948.py:34: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  data = json.dumps(completion.choices[0].message.parsed.dict())
